# Recommendation system
The idea of the project is to predict ratings of unwatched movies  for userId 610 in order to be able to recommend movies to this user


### Data reading

In [264]:
import pandas as pd
import numpy as np
np.random.seed(156068)
from scipy.stats import pearsonr

df = pd.read_csv('./ratings.csv')
df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


### Transforming the dataframe to the convinient form

In [265]:
userId = 610
transformed_df = df.pivot(index='movieId', columns='userId', values='rating')
transformed_df

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Coorelation finding methods
Here there are the functions used for finding the coorelations between the users basing on the mutual movies that  they have watched 

In [ ]:
'''the coorelation finding method which just finds the pearson coorelation between the  given user and all other users , 
if they have at least 2 movies in common
and if neither given user nor other user ratings for common films are constant array '''
def find_correlation(transformed_df:pd.DataFrame,userId=610):
    user_column = transformed_df[userId].copy()
    user_column.dropna(inplace=True)
    correlations = dict()
    


    for other_user in transformed_df.drop(userId,axis=1).columns:
        #cannot check correlations between arrays with 1 element 
        common_ratings = user_column.index.intersection(transformed_df[other_user].dropna().index)
        if len(common_ratings)>1: 
            common_ratings = list(common_ratings) 
            
            # cannot check correlation with array of all elements with the same value, since the concept of  correlation does not apply there
            if user_column[common_ratings].nunique() > 1 and transformed_df[other_user][common_ratings].nunique() > 1:
                corr, _ = pearsonr(user_column[common_ratings], transformed_df[other_user][common_ratings])
                correlations[other_user] = corr
                continue 

        correlations[other_user] = np.nan  


    correlation_df = pd.Series(correlations)

    return correlation_df


'''the coorelation finding function as above,
 however  the coorelation is found only if the user and other user have at least threshold films in common'''
def find_correlation_with_common_films_threshold(transformed_df:pd.DataFrame,threshold=5,userId=610):
    user_column = transformed_df[userId].copy()
    user_column.dropna(inplace=True)
    correlations = dict()
    


    for other_user in transformed_df.drop(userId,axis=1).columns:
        #cannot check correlations between arrays with 1 element 
        common_ratings = user_column.index.intersection(transformed_df[other_user].dropna().index)
        if len(common_ratings)>threshold: 
            common_ratings = list(common_ratings) 
            # cannot check correlation with array of all elements with the same value, since the concept of  correlation does not apply there
            if user_column[common_ratings].nunique() > 1 and transformed_df[other_user][common_ratings].nunique() > 1:
                corr, _ = pearsonr(user_column[common_ratings], transformed_df[other_user][common_ratings])
                correlations[other_user] = corr
                continue 

        correlations[other_user] = np.nan  

    correlation_df = pd.Series(correlations)
    


    return correlation_df

### Movie recommender systems
Here can be found the different movie recommender systems approaches

In [ ]:

def predict_scores(transformed_df: pd.DataFrame, userId=610, k=10,split = None):
    '''we find up to k most coorelated users with our userID  that have rated given movie not seen by our userID
        and calculate rating  as a weighted average
        of their ratings for this film. up to k because it may happen that there are less than k users 
        with positive correlation with our userId who watched
        given movie'''
    transposed_df = transformed_df.transpose()
    user_correlations = find_correlation(transformed_df, userId)
    sorted_correlations = user_correlations.sort_values(ascending=False)
    sorted_correlations = sorted_correlations[sorted_correlations > 0]
    sorted_transposed_df = transposed_df.loc[sorted_correlations.index]
    if userId not in sorted_transposed_df.index:
        sorted_transposed_df.loc[userId] = transposed_df.loc[userId]
    predictions_df = sorted_transposed_df.copy()

 
    for movie in sorted_transposed_df.columns:
        ratings_for_movie = sorted_transposed_df[movie]
        if split is not None:
            if movie not in split:
                continue
       
        if pd.isna(ratings_for_movie[userId]):
            valid_ratings = ratings_for_movie.dropna()
            top_users_indices = sorted_correlations.index.intersection(valid_ratings.index)[:k]
            if top_users_indices.empty:
                continue
            weighted_avg = valid_ratings[top_users_indices].dot(user_correlations[top_users_indices]) / user_correlations[top_users_indices].sum()
            valid_score = round(weighted_avg * 2) / 2
            predictions_df.loc[userId, movie] = valid_score

    
    return predictions_df.transpose()




def predict_scores_no_baseline(
    transformed_df: pd.DataFrame,
    userId=610,
    k=10,
    
    split=None
):
    """
    Predict scores using top `k` most correlated users. Rating for tuple (userId,movie) predicted as mean of ratings for our userID for all movies + 
    weighted deviation from the mean for given movie of the k most correlated users.
    """
    transposed_df = transformed_df.transpose()
    user_correlations = find_correlation(transformed_df, userId)
    sorted_correlations = user_correlations.sort_values(ascending=False)
    user_means = transposed_df.mean(axis=1, skipna=True)
    user_mean_target = user_means[userId]
    predictions_df = transposed_df.copy()

    for movie in transposed_df.columns:
        if split is not None and movie not in split:
            continue
        ratings_for_movie = transposed_df[movie]
        if pd.isna(ratings_for_movie[userId]):
            valid_ratings = ratings_for_movie.dropna()
            valid_users_sorted = sorted_correlations.index.intersection(valid_ratings.index)
            top_k_users = valid_users_sorted[:k]
            if len(top_k_users) == 0:
                continue
            weighted_deviation_sum = 0
            total_weight = 0

            for user in top_k_users:
                correlation = sorted_correlations[user]  
                user_mean = user_means[user]  
                user_rating = ratings_for_movie[user] 
                deviation = user_rating - user_mean
                weighted_deviation_sum += correlation * deviation
                total_weight += abs(correlation)
            if total_weight > 0:
                weighted_deviation = weighted_deviation_sum / total_weight
                predicted_score = user_mean_target + weighted_deviation
                predictions_df.loc[userId, movie] = predicted_score
    return predictions_df.transpose()


def predict_scores_with_item_correction(
    transformed_df: pd.DataFrame,
    userId=610,
    k=10,
    border  =0.8,
    split=None
):
    """
    works similar to the predictor above, only at the end it adjusts the  rating basing on the mean rating for given film in case
    the rating is close to the border of ratings 
    """
    transposed_df = transformed_df.transpose()
    user_correlations = find_correlation(transformed_df, userId)
    sorted_correlations = user_correlations.sort_values(ascending=False)
    user_means = transposed_df.mean(axis=1, skipna=True)
    item_means = transformed_df.mean(axis=1, skipna=True)
    user_mean_target = user_means[userId]
    predictions_df = transposed_df.copy()

    for movie in transposed_df.columns:
        if split is not None and movie not in split:
            continue
        ratings_for_movie = transposed_df[movie]
        if pd.isna(ratings_for_movie[userId]):
            valid_ratings = ratings_for_movie.dropna()
            valid_users_sorted = sorted_correlations.index.intersection(valid_ratings.index)
            top_k_users = valid_users_sorted[:k]
            if len(top_k_users) == 0:
                continue
            weighted_deviation_sum = 0
            total_weight = 0

            for user in top_k_users:
                correlation = sorted_correlations[user]  
                user_mean = user_means[user]  
                user_rating = ratings_for_movie[user]  
                deviation = user_rating - user_mean
                weighted_deviation_sum += correlation * deviation
                total_weight += abs(correlation)
            
            weighted_deviation = weighted_deviation_sum / total_weight if total_weight > 0 else 0
            item_mean = item_means[movie] if movie in item_means else 0
            if item_mean >= round(user_mean_target*2)/2  and 2*user_mean_target- int(2*user_mean_target) >= border:
                correction = 0.5
            elif item_mean <= round(user_mean_target*2)/2  and 2*user_mean_target- int(2*user_mean_target) <= (1-border):
                correction = - 0.5
            else:
                correction = 0

            predicted_score = user_mean_target + weighted_deviation + correction
            predicted_score = round(predicted_score * 2) / 2  
            predicted_score = max(0.5,predicted_score)
            predictions_df.loc[userId, movie] = predicted_score

    return predictions_df.transpose()





def predict_baseline(transformed_df: pd.DataFrame, userId=610, k=1,split=None):
    """
    Predict baseline scores for movies the user has not rated.
    mean of user ratings of this user + movie ratings for this movie
    """
    predictions_df = transformed_df.copy()
    for movie in transformed_df.index:
        if split is not None and movie not in split:
            continue

        if pd.isna(transformed_df.loc[movie, userId]):
            user_ratings = transformed_df[userId].dropna()
            movie_ratings = transformed_df.loc[movie].dropna()
            total_ratings_sum = user_ratings.sum() + movie_ratings.sum()
            total_ratings_count = len(user_ratings) + len(movie_ratings)
            if total_ratings_count > 0:
                baseline_prediction = total_ratings_sum / total_ratings_count
                predictions_df.loc[movie, userId] = round(baseline_prediction * 2) / 2  # Round to nearest 0.5

    return predictions_df







def predict_scores_with_threshold(
    transformed_df: pd.DataFrame,
    userId=610,
    k=1,
    threshold=0.8,
    split=None
):
    """
    Predict scores using all correlated users above a specified threshold.
    weighted average of these users  rating for this film . Weights are the coorelation
    
    """
    transposed_df = transformed_df.transpose()
    user_correlations = find_correlation(transformed_df, userId)
    filtered_correlations = user_correlations[user_correlations > threshold]
    sorted_transposed_df = transposed_df.loc[filtered_correlations.index]
    if userId not in sorted_transposed_df.index:
        sorted_transposed_df.loc[userId] = transposed_df.loc[userId]

    predictions_df = sorted_transposed_df.copy()
    for movie in sorted_transposed_df.columns:
        ratings_for_movie = sorted_transposed_df[movie]
        if split is not None:
            if movie not in split:
                continue
        if pd.isna(ratings_for_movie[userId]):
            valid_ratings = ratings_for_movie.dropna()
            valid_users_indices =filtered_correlations.index.intersection(valid_ratings.index)
            if valid_users_indices.empty:
                continue
            weighted_avg = (
                valid_ratings[valid_users_indices].dot(filtered_correlations[valid_users_indices]) /
                filtered_correlations[valid_users_indices].sum()
            )

            valid_score = round(weighted_avg * 2) / 2
            predictions_df.loc[userId, movie] = valid_score
    return predictions_df.transpose()


def predict_scores_with_threshold_negative(
    transformed_df: pd.DataFrame,
    userId=610,
    k=1,
    positive_threshold=0,
    negative_threshold=0,
    split=None
):
    """
    Predict scores using all correlated users above a specified threshold for positive correlations,
    and include negatively correlated users below a specified threshold with adjusted contribution.
    """
    transposed_df = transformed_df.transpose()
    user_correlations = find_correlation(transformed_df, userId)
    positive_correlations = user_correlations[user_correlations > positive_threshold]
    negative_correlations = user_correlations[
        (user_correlations < 0) & (user_correlations.abs() > negative_threshold)
    ]
    sorted_transposed_df = transposed_df.loc[
        positive_correlations.index.union(negative_correlations.index)
    ]
    if userId not in sorted_transposed_df.index:
        sorted_transposed_df.loc[userId] = transposed_df.loc[userId]
    predictions_df = sorted_transposed_df.copy()

    for movie in sorted_transposed_df.columns:
        ratings_for_movie = sorted_transposed_df[movie]

        if split is not None:
            if movie not in split:
                continue

        if pd.isna(ratings_for_movie[userId]):
            valid_ratings = ratings_for_movie.dropna()
            positive_users = positive_correlations.index.intersection(valid_ratings.index)
            negative_users = negative_correlations.index.intersection(valid_ratings.index)
            weighted_sum = 0
            total_weight = 0
            if not positive_users.empty:
                weighted_sum += (
                    valid_ratings[positive_users].dot(positive_correlations[positive_users])
                )
                total_weight += positive_correlations[positive_users].sum()

            for neg_user in negative_users:
                neg_rating = valid_ratings[neg_user]
                if neg_rating < 2 or neg_rating > 4: 
                    adjusted_rating = 6 - neg_rating
                    weight = abs(negative_correlations[neg_user])
                    weighted_sum += adjusted_rating * weight
                    total_weight += weight

            if total_weight == 0:
                continue

            weighted_avg = weighted_sum / total_weight
            valid_score = round(weighted_avg * 2) / 2
            predictions_df.loc[userId, movie] = valid_score
    return predictions_df.transpose()











def predict_scores_strict_correlation(transformed_df: pd.DataFrame, userId=610, k=10,correlation_neighbours=5,split = None):

    '''function working the same as predict_scores described above, with only difference , that coorelation are defined  only for the users
    having at least coorelation_neighbours mutual watched films with userId '''
    transposed_df = transformed_df.transpose()
    user_correlations = find_correlation_with_common_films_threshold(transformed_df,correlation_neighbours,userId)
    sorted_correlations = user_correlations.sort_values(ascending=False)
    sorted_correlations = sorted_correlations[sorted_correlations > 0]
    sorted_transposed_df = transposed_df.loc[sorted_correlations.index]
    if userId not in sorted_transposed_df.index:
        sorted_transposed_df.loc[userId] = transposed_df.loc[userId]
    predictions_df = sorted_transposed_df.copy()
    for movie in sorted_transposed_df.columns:
        if split is not None:
            if movie not in split:
                continue
        ratings_for_movie = sorted_transposed_df[movie]
        if pd.isna(ratings_for_movie[userId]):
            valid_ratings = ratings_for_movie.dropna()
            top_users_indices = sorted_correlations.index.intersection(valid_ratings.index)[:k]
            if top_users_indices.empty:
                continue
            weighted_avg = valid_ratings[top_users_indices].dot(user_correlations[top_users_indices]) / user_correlations[top_users_indices].sum()
            valid_score = round(weighted_avg * 2) / 2
            predictions_df.loc[userId, movie] = valid_score
    return predictions_df.transpose()














### Evaluation of the performance of predicton methods

Frameworks for evaluating the recommender systems. All of them are based on the 10-fold cross validation on the userID 610, ensuring the splits are always the same across every run of evaluation methods , so  that the comparison of different recommender systems is plausible.

In [ ]:
def evaluate_recommendation_quality_general(
    transformed_df,
    predict_function,  
    userId=610,
    k=10,
    num_splits=10,
    **predict_function_kwargs  
):
    """
    General evaluation function for recommendation quality using Mae. 
    """
    original_ratings = transformed_df.loc[:, userId].dropna()
    rated_movies = original_ratings.index  
    splits = np.array_split(rated_movies, num_splits)
    mae_list = []
    for split in splits:
        df_with_nan = transformed_df.copy()
        df_with_nan.loc[split, userId] = np.nan
        predicted_df = predict_function(df_with_nan, userId=userId, k=k, split=split, **predict_function_kwargs)
        predicted_scores = predicted_df.loc[split, userId]
        actual_scores = original_ratings[split]
        valid_indices = ~np.isnan(predicted_scores) & ~np.isnan(actual_scores)  
        mae = np.abs(predicted_scores[valid_indices] - actual_scores[valid_indices]).mean() 
        
        mae_list.append(mae)
      
    
    return np.mean(mae_list)

def evaluate_recommendation_quality_at_p(
    transformed_df,
    predict_function,  
    userId=610,
    k=10,
    p=10,
    num_splits=10,
    **predict_function_kwargs  
):
    """
    Function for evaluating the Mae at p  highest rated items.
    """
    original_ratings = transformed_df.loc[:, userId].dropna()
    rated_movies = original_ratings.index  
    splits = np.array_split(rated_movies, num_splits)
    mae_list = []

    for split in splits:
        df_with_nan = transformed_df.copy()
        df_with_nan.loc[split, userId] = np.nan
        predicted_df = predict_function(df_with_nan, userId=userId, k=k, split=split, **predict_function_kwargs)
        predicted_scores = predicted_df.loc[split, userId]
        actual_scores = original_ratings[split]
        predicted_scores.sort_values(ascending=False)
        predicted_scores = predicted_scores[:p]
        valid_indices = ~np.isnan(predicted_scores) & ~np.isnan(actual_scores)  
        mae = np.abs(predicted_scores[valid_indices] - actual_scores[valid_indices]).mean()  
        mae_list.append(mae)
    return np.mean(mae_list)

def evaluate_recommendation_quality_rmse_general(
    transformed_df,
    predict_function,  
    userId=610,
    k=10,
    num_splits=10,
    **predict_function_kwargs  
):
    """
    General evaluation function for recommendation quality using RMSE.
    """
    
    original_ratings = transformed_df.loc[:, userId].dropna()
    rated_movies = original_ratings.index  
    splits = np.array_split(rated_movies, num_splits)
    rmse_list = []

    for split in splits:
        df_with_nan = transformed_df.copy()
        df_with_nan.loc[split, userId] = np.nan
        predicted_df = predict_function(df_with_nan, userId=userId, k=k, split=split, **predict_function_kwargs)
        predicted_scores = predicted_df.loc[split, userId]
        actual_scores = original_ratings[split]
        valid_indices = ~np.isnan(predicted_scores) & ~np.isnan(actual_scores) 
        rmse = np.sqrt(((predicted_scores[valid_indices] - actual_scores[valid_indices]) ** 2).mean())  
        rmse_list.append(rmse)
    return np.mean(rmse_list)



def evaluate_recommendation_quality_general_weighted(
transformed_df,predict_function,
 weight_under=1,weight_over=1,
 userId=610, k=20, num_splits=10,
**predict_function_kwargs):
    '''function used to evalue the overestimation error and underestimation error'''
    original_ratings = transformed_df.loc[:, userId].dropna()
    rated_movies = original_ratings.index 
    splits = np.array_split(rated_movies, num_splits)
    mae_list = []
    for split in splits:
        df_with_nan = transformed_df.copy()
        df_with_nan.loc[split, userId] = np.nan
        predicted_df = predict_function(df_with_nan,userId=userId, k=k, split=split,**predict_function_kwargs)
        predicted_scores = predicted_df.loc[split, userId]
        actual_scores = original_ratings[split]
        valid_indices = ~np.isnan(predicted_scores) & ~np.isnan(actual_scores)  
        error_array = predicted_scores[valid_indices] - actual_scores[valid_indices]
        underpredicted_error = error_array[error_array<=0]
        overpredicted_error = error_array[error_array>0]
        underpredicted_error = np.abs(underpredicted_error)
        underpredicted_error  = underpredicted_error * weight_under
        overpredicted_error = overpredicted_error * weight_over
        weighted_mae = (sum(underpredicted_error) + sum(overpredicted_error)) / (len(underpredicted_error)*weight_under + len(overpredicted_error)*weight_over)
        mae_list.append(weighted_mae)
    return np.mean(mae_list)

### Description of testing 

We decided to test several movie recommender systems, which will be duly described in each step, as well as  the conclusions of the results  of the  given  systems. In order to test the predicition accuracy of the system we have devised several different evaluation methods. All of them are basing on the 10-fold  cross validation on the userId 610. It means we run 10 iterations of the recommender systems, in each giving it 9/10 of the records for userID 610 and all the records for other users and we command it to predict score for the deliberately erased data for userId 610 . We calculate the error and take the mean of the error for the 10 runs. Surely we would get more data if we tried to do the same approach with several other users, but since our task is specifically to predict the ratings for the movies for userId 610 , we base the evaluation only on the algorithms error on this userID. When  creating these splits for 10-fold validation we do not shuffle the data as it is not anyhow sorted byt rating for the Userid 610 and we would want to always evaluate the recommender systems on the exactly same splits to compare them as good as possible. As we have  mentioned there are several methods of error evaluations, mainly:
- MAE , simply the mean absolute error , very easy to interpret measure
- MRSE, the mean root squared error, a good measure for this specific purpose, as it penalizes biggger mistakes more
- Weighted MAE , the idea is that  our user surely will have ceratain preference. The overestimating may be more detrimental to the user (if he is very picky or  doesnt have much time for watching the movies) then what matters more is the precision of high ratings. On the other hand if the user spends all his day watching films, he probably would not really care  much for the false positive, but on the other hand, he would not want to run out of movie
choices, therefore false negative high rating is bad for him. So to summ up , we do the mae but we set weight to the overestimation error and weight to 
underestimation erorr, and we can alter these weights to see best  systems for high precision in high ratings and best  systems for high recall for high ratings. To have only one type of the error accounted for we will use weights (1,0) and (0,1), so  accounting only for one type of error.
- Lastly we have a mea but only for the k  best rated films . Since surely user may not care that much about the ratings of  the films he wouldnt watch anyway , it is maybe  somehow similar to the previous approach overestimation error, since it would work good for a picky user with small amount of time - he doesnt want to watch some crap, but wont care that much about ratings of films he wont have time to watch anyway.



# Testing

##### remark
- What we have not mentioned explicitly earlier , each system when having final prediction will round it to the closest viable value, because we want to obtain correct prediction in  the given scale i.e  for the ratings.csv from 0.5 to 5 with change by 0.5
- when referring to the correlation between users, we always refer to the pearson coorelation

In [269]:
### we will store best errors, to be able later to seem which system performed best for which purpose
mae_list = []
rmse_list = []
perror_list = []
overestimate_error_list = []
underestimate_error_list = []

### Baseline predictor
Firstly we would like to check the performance of the baseline predictor, to have something to compare with later. The baseline predictor is a very simple approach which  when predicting score for given tuple (movie,userID) returns  mean of all the ratings for given movie + all the ratings for given userID 

In [270]:

mae = evaluate_recommendation_quality_general(
        transformed_df=transformed_df,
        predict_function=predict_baseline,  # Function to predict scores
        userId=610,
        num_splits=10 # Pass the threshold as a keyword argument
    )
mae_list.append(('predict_baseline',mae))
rmse = evaluate_recommendation_quality_rmse_general(
        transformed_df=transformed_df,
        predict_function=predict_baseline,  # Function to predict scores
        userId=610,
        num_splits=10 # Pass the threshold as a keyword argument
    )
print(f"MAE using the baseline predictio {mae}")
print(f"RMSE using the baseline predictio {rmse}")
rmse_list.append(('predict_baseline',rmse))

p_error = evaluate_recommendation_quality_at_p(
        p=10,
        transformed_df=transformed_df,
        predict_function=predict_baseline,  # Function to predict scores
        userId=610,
        num_splits=10 # Pass the threshold as a keyword argument
    )
print(f"Mae at p=10 using the baseline prediction {p_error}")
perror_list.append(('predict_baseline',p_error))


overestimate_error = evaluate_recommendation_quality_general_weighted(
        transformed_df=transformed_df,
        weight_under=0,
        weight_over=1,
        predict_function=predict_baseline,  # Function to predict scores
        userId=610,
        num_splits=10 # Pass the threshold as a keyword argument
    )
print(f"Overestimate MAE is {overestimate_error} ")
overestimate_error_list.append(('predict_baseline',overestimate_error))

underestimate_error = evaluate_recommendation_quality_general_weighted(
        transformed_df=transformed_df,
        weight_under=1,
        weight_over=0,
        userId=610,
        predict_function=predict_baseline,  # Function to predict scores
        num_splits=10 # Pass the threshold as a keyword argument
    )
print(f"Underestimate  MAE is {underestimate_error} ")
underestimate_error_list.append(('predict_baseline',underestimate_error))



MAE using the baseline predictio 0.6701644157369347
RMSE using the baseline predictio 0.8670309338053681
Mae at p=10 using the baseline prediction 0.675
Overestimate MAE is 0.8334233572346182 
Underestimate  MAE is 0.5890073252784332 


#### conclusions
Honestly it does not  perform that terribly in terms of the mae , it can be a good computationaly cheap start. One thing is that this  system, at least for this data, turned out quite biased, overestimating error is significantly higher than the underestimating error, so it tends to overestimate.

### user coorelation approaches
now , as we have something to compare our more sophisticated  recommender systems with , we can start to test them. The first system  sets the prediction for tuple (userID,movie) as weighted average of ratings for  given movie for users which watched  given movie and had the positive correlation with the given user with value of the coorelation above some threshold. we will check several corrrelation thresholds above which we include the users to the weighted mean and see which one performs the best . 

In [ ]:
for correlation_threshold in [0, 0.2, 0.4, 0.6]:
        mae = evaluate_recommendation_quality_general(
            transformed_df=transformed_df,
            num_splits=10,
            predict_function=predict_scores_with_threshold,
            userId=610,
            threshold=correlation_threshold 
        )
        rmse = evaluate_recommendation_quality_rmse_general(
                transformed_df=transformed_df,
                predict_function=predict_scores_with_threshold,  
                userId=610,
                num_splits=10,
                threshold=correlation_threshold 
            )
        print(f"MAE using {correlation_threshold} positive coorelation threshold {mae}")
        mae_list.append((f"MAE using {correlation_threshold}% positive coorelation threshold",mae))
        
        print(f"RMSE using {correlation_threshold} positive coorelation threshold {rmse}")
        rmse_list.append((f"Rmse using {correlation_threshold}% positive coorelation threshold",rmse))

        p_error = evaluate_recommendation_quality_at_p(
                p=10,
                transformed_df=transformed_df,
                num_splits=10,
                predict_function=predict_scores_with_threshold,  
                userId=610,
                threshold=correlation_threshold 
            )
        print(f"Mae at p=10 using {correlation_threshold}% positive coorelation threshold {p_error}")
        perror_list.append((f"Mae at p=10 using {correlation_threshold} positive coorelation threshold",p_error))

        overestimate_error = evaluate_recommendation_quality_general_weighted(
                transformed_df=transformed_df,
                weight_under=1,
                weight_over=2,
                userId=610,
                num_splits=10,
                predict_function=predict_scores_with_threshold,  
                threshold=correlation_threshold 
            )
        print(f"Overestimate MAE is {overestimate_error} ")
        overestimate_error_list.append((f"Overestimate Mae using  {correlation_threshold}% positive coorelation threshold",overestimate_error))

        underestimate_error = evaluate_recommendation_quality_general_weighted(
                transformed_df=transformed_df,
                weight_under=1,
                weight_over=0,
                predict_function=predict_scores_with_threshold,  
                userId=610,
                num_splits=10,
                threshold=correlation_threshold 
            )
        print(f"underestimate_error using {correlation_threshold}% positive coorelation threshold {underestimate_error}")
        underestimate_error_list.append((f"p_error using {correlation_threshold}% positive coorelation threshold",underestimate_error))


MAE using 0 positive coorelation threshold 0.6021952781071522
RMSE using 0 positive coorelation threshold 0.7939195306055609
Mae at p=10 using 0% positive coorelation threshold 0.505952380952381
Overestimate MAE is 0.6211578619909098 
underestimate_error using 0% positive coorelation threshold 0.574598358638441
MAE using 0.2 positive coorelation threshold 0.606241746389952
RMSE using 0.2 positive coorelation threshold 0.802867162923574
Mae at p=10 using 0.2% positive coorelation threshold 0.5147222222222223
Overestimate MAE is 0.619356961510738 
underestimate_error using 0.2% positive coorelation threshold 0.5873767604119139
MAE using 0.4 positive coorelation threshold 0.6355510566787939
RMSE using 0.4 positive coorelation threshold 0.8408924823724673
Mae at p=10 using 0.4% positive coorelation threshold 0.5407936507936507
Overestimate MAE is 0.6449831749158503 
underestimate_error using 0.4% positive coorelation threshold 0.6224698281566787
MAE using 0.6 positive coorelation threshold

#### conclusions
It has some advantages compared  to the baseline approach , however the difference considering mae is not staggering, about ~10% . The Rmse is  decreasing even by lower percentage. However  the predictions are definitely more balanced, i.e the difference between the underestimation_error and over_estimation_error is neglibile in comparison to what it was previously.  Moreover the error at p=10 best ratings decreases visibly, what might be important  in reall recommender systems since, these are the  movies the user would watch if he was to choose by predicted rating. 
Interestingly, in this case in nearly all metrics,  setting threshold to 0 therefore taking all positively correlated users proved to give the best effects. On the other hand creating the score only basing on users with positive coorelation > 0.6 could even be worse than the basline approach as the Rmse is bigger here! It could be probably casued by the scarcity of data, as it could occur that these users with high coorelation had for example only 3 films in common with our userId 610 , therefore were not very 'reliable' source of information 

### using the negatively coorelated
since we do not suffer from abundance of data, it would be great to take adventage of the users which are negatively coorelated to the current user. There is plenty of such  users and we devised an algorithm which can do that. Mainly we will use the positive coorelation as in the previous function, but then we will also check for the negatively coorelated users with the magnitude of the coorelation above some threshold. i.e we want to have strongly negatively coorelated users. Then we  incorporate them in the weighted average , but in doing so , we do not take their ratings as they are, but we change them to rating = 6-rating. Why? because since the relation is negative, if these users rated  given film high, we expect our user to rate it low , so in other words we create sort of a complement of a rating. Also we do that only  for ratings of strongly negatively correalated users in range (0,2> and <4,5>) since for example  if we have strongly negatively correlated user who give 3 for some movie , we would have to give our use also 3 ->6-3 = 3 , but it seems it shall not work this way

In [272]:
for negative_correlation in [0.2,0.4,0.6]:
        mae = evaluate_recommendation_quality_general(
            transformed_df=transformed_df,
            num_splits=10,
            predict_function=predict_scores_with_threshold_negative,  # Function to predict scores
            userId=610,
            positive_threshold = 0 ,
            negative_threshold = negative_correlation
             # Pass the threshold as a keyword argument
        )
        rmse = evaluate_recommendation_quality_rmse_general(
                transformed_df=transformed_df,
                predict_function=predict_scores_with_threshold_negative,  # Function to predict scores
                userId=610,
                num_splits=10,
                positive_threshold = 0 ,
                negative_threshold = negative_correlation
            )
        print(f"MAE using 0% positive and  {correlation_threshold}%  negative  coorelation threshold {mae}")
        mae_list.append((f"MAE using 0% positive and  {correlation_threshold}%  negative  coorelation threshold",mae))
        
        print(f"RMSE using 0% positive and  {correlation_threshold}%  negative  coorelation threshold {rmse}")
        rmse_list.append((f"Rmse using {correlation_threshold} positive coorelation threshold",rmse))

        p_error = evaluate_recommendation_quality_at_p(
                p=10,
                transformed_df=transformed_df,
                num_splits=10,
                predict_function=predict_scores_with_threshold_negative,  # Function to predict scores
                userId=610,
                positive_threshold = 0 ,
                negative_threshold = negative_correlation
            )
        print(f"Ma at p=10 using 0% positive and  {correlation_threshold}%  negative  coorelation threshold{p_error}")
        perror_list.append((f"p_error using 0% positive and  {correlation_threshold}%  negative  coorelation threshold",p_error))

        overestimate_error = evaluate_recommendation_quality_general_weighted(
                transformed_df=transformed_df,
                weight_under=0,
                weight_over=1,
                userId=610,
                num_splits=10,
                predict_function=predict_scores_with_threshold_negative,  # Function to predict scores
                positive_threshold = 0 ,
                negative_threshold = negative_correlation
            )
        print(f"overestimate_error using 0% positive and  {correlation_threshold}%  negative  coorelation threshold {overestimate_error} ")
        overestimate_error_list.append((f"overestimate_erro using 0% positive and  {correlation_threshold}%  negative  coorelation threshold",overestimate_error))
        underestimate_error = evaluate_recommendation_quality_general_weighted(
                transformed_df=transformed_df,
                weight_under=1,
                weight_over=0,
                predict_function=predict_scores_with_threshold_negative,  # Function to predict scores
                userId=610,
                num_splits=10,
                positive_threshold = 0,
                negative_threshold = negative_correlation
            )
        print(f"underestimate_error using 0% positive and  {correlation_threshold}%  negative  coorelation threshold {underestimate_error}")
        underestimate_error_list.append((f"underestimate_error using 0% positive and  {correlation_threshold}%  negative  coorelation threshold",underestimate_error))

MAE using 0% positive and  0.6%  negative  coorelation threshold 0.6113749808614168
RMSE using 0% positive and  0.6%  negative  coorelation threshold 0.80136603031125
Ma at p=10 using 0% positive and  0.6%  negative  coorelation threshold0.510952380952381
overestimate_error using 0% positive and  0.6%  negative  coorelation threshold 0.745418158624428 
underestimate_error using 0% positive and  0.6%  negative  coorelation threshold 0.5859043019335467
MAE using 0% positive and  0.6%  negative  coorelation threshold 0.6060534511840754
RMSE using 0% positive and  0.6%  negative  coorelation threshold 0.7972685194076556
Ma at p=10 using 0% positive and  0.6%  negative  coorelation threshold0.510952380952381
overestimate_error using 0% positive and  0.6%  negative  coorelation threshold 0.7347326854718496 
underestimate_error using 0% positive and  0.6%  negative  coorelation threshold 0.5793087606016811
MAE using 0% positive and  0.6%  negative  coorelation threshold 0.604099744608393
RMSE

### conclusions
in fact it didnt seem to work out as planned as it worsen the performance in comparison to the previous algorithm  in all of the metrics. Therefore we shall abandon this idea, anyway it was worth checking out definitely.

### fixed neighbourhood size of k most similar neighbours
The idea with the negative coorelations didnt really work therefore  we want to further explore the user-based rating prediction basing on the positively coorelated users, hovever this time we wont take all users above some threshold but instead the k most similar users with our UserId and we will create the weighted average of them

In [273]:
for neighbours_size in [ 5,15,25,50]:
        mae = evaluate_recommendation_quality_general(
            transformed_df=transformed_df,
            num_splits=10,
            predict_function=predict_scores,  # Function to predict scores
            userId=610,
            k = neighbours_size
             # Pass the threshold as a keyword argument
        )
        rmse = evaluate_recommendation_quality_rmse_general(
                transformed_df=transformed_df,
                predict_function=predict_scores,  # Function to predict scores
                userId=610,
                num_splits=10,
                k = neighbours_size
            )
        print(f"MAE using {neighbours_size} strongest positively coorelated neighbours {mae}")
        mae_list.append((f"MAE using {neighbours_size} strongest positively coorelated neighbours",mae))
        
        print(f"RMSE using {neighbours_size} strongest positively coorelated neighbours {rmse}")
        rmse_list.append((f"Rmse using {neighbours_size} strongest positively coorelated neighbours",rmse))

        p_error = evaluate_recommendation_quality_at_p(
                p=10,
                transformed_df=transformed_df,
                num_splits=10,
                predict_function=predict_scores,  # Function to predict scores
                userId=610,
                k = neighbours_size
            )
        print(f"Ma at p=10 using {neighbours_size} strongest positively coorelated neighbours {p_error}")
        perror_list.append((f"p_error using {neighbours_size} strongest positively coorelated neighbours",p_error))

        overestimate_error = evaluate_recommendation_quality_general_weighted(
                transformed_df=transformed_df,
                weight_under=0,
                weight_over=1,
                userId=610,
                num_splits=10,
                predict_function=predict_scores,  # Function to predict scores
                k = neighbours_size
            )
        print(f"overestimate_error using  {neighbours_size} strongest positively coorelated neighbours  {overestimate_error} ")
        overestimate_error_list.append((f"overestimate_error using {neighbours_size} strongest positively coorelated neighbours",overestimate_error))
        underestimate_error = evaluate_recommendation_quality_general_weighted(
                transformed_df=transformed_df,
                weight_under=1,
                weight_over=0,
                predict_function=predict_scores,  # Function to predict scores
                userId=610,
                num_splits=10,
                k = neighbours_size
            )
        print(f"underestimate_error using  {neighbours_size} strongest positively coorelated neighbours  {underestimate_error}")
        underestimate_error_list.append((f"underestimate_error using {neighbours_size} strongest positively coorelated neighbours",underestimate_error))
 

MAE using 5 strongest positively coorelated neighbours 0.6189454443760426
RMSE using 5 strongest positively coorelated neighbours 0.8134962988471877
Ma at p=10 using 5 strongest positively coorelated neighbours 0.543015873015873
overestimate_error using  5 strongest positively coorelated neighbours  0.7107882585333297 
underestimate_error using  5 strongest positively coorelated neighbours  0.5964032868467976
MAE using 15 strongest positively coorelated neighbours 0.6031413821118339
RMSE using 15 strongest positively coorelated neighbours 0.7939928210640812
Ma at p=10 using 15 strongest positively coorelated neighbours 0.4970634920634921
overestimate_error using  15 strongest positively coorelated neighbours  0.7272806141227194 
underestimate_error using  15 strongest positively coorelated neighbours  0.5787871031982827
MAE using 25 strongest positively coorelated neighbours 0.5981621909345929
RMSE using 25 strongest positively coorelated neighbours 0.7906557039685967
Ma at p=10 using 

#### conclusions
It seems to have yielded more promising results than the previous idea of the negative coorelations incorporation. However the improvements are not anyhow staggering. The RMSE did not improve in comparison to taking all positively coorelated users , however the mae and mae at p=10  have now improved , but it also depends on the size of the neighbourhood, so this parameter must be choosen wisely. Overall the best performance have been achieved with 25 most similar users. Unfortunately, maybe due to taking less amount of data into consideration when predictiong rating for each film , the overestimation_error and underestimation error differ quite a lot -the algorithm tends to overestimate , it  could also be beneficial for a user who has a lot of time for watching movies and would not mind some false positive high ratings, but in general it is not the best outcome.

### change of approach
So until now we focused on the coorelation of our user to the other users and ratings for these other users for this film, it seems resonable, but in fact it does not account for one important thing: The personal tendency of users to rate films high or low . So some users may be grumpy in general whereas others might be indulgent overall in their grading, so to account for that we need to look how the rating for the film that we are currently considering deviates for the given users from their rating means, this way we account for the individual tendencies to rank high or low. Therefore our recommender system shall be in fact more versatile and accurate. We will use previously discovered threshold of 25 strongest positively correlated users in this case as well. 

In [274]:
mae = evaluate_recommendation_quality_general(
            transformed_df=transformed_df,
            num_splits=10,
            predict_function=predict_scores_no_baseline,  # Function to predict scores
            userId=610,
            k = 25
             # Pass the threshold as a keyword argument
        )
rmse = evaluate_recommendation_quality_rmse_general(
                transformed_df=transformed_df,
                predict_function=predict_scores_no_baseline,  # Function to predict scores
                userId=610,
                num_splits=10,
                k = 25
            )
print(f"MAE using 25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low {mae}")
mae_list.append((f"MAE using 25strongest positively coorelated neighbours accounting for the personal  tendencies high/low",mae))
        
print(f"RMSE using 25 strongest positively coorelated neighbours  accounting for the personal  tendencies high/low{rmse}")
rmse_list.append((f"Rmse using 25strongest positively coorelated neighbours accounting for the personal  tendencies high/low",rmse))

p_error = evaluate_recommendation_quality_at_p(
                p=10,
                transformed_df=transformed_df,
                num_splits=10,
                predict_function=predict_scores_no_baseline,  # Function to predict scores
                userId=610,
                k = 25
            )
print(f"Ma at p=10 using 25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low {p_error}")
perror_list.append((f"p_error using 25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low",p_error))

overestimate_error = evaluate_recommendation_quality_general_weighted(
                transformed_df=transformed_df,
                weight_under=0,
                weight_over=1,
                userId=610,
                num_splits=10,
                predict_function=predict_scores_no_baseline,  # Function to predict scores
                k = 25
            )
print(f"overestimate_error using  25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low  {overestimate_error} ")
overestimate_error_list.append((f"overestimate_error using 25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low",overestimate_error))
underestimate_error = evaluate_recommendation_quality_general_weighted(
                transformed_df=transformed_df,
                weight_under=1,
                weight_over=0,
                predict_function=predict_scores_no_baseline,  # Function to predict scores
                userId=610,
                num_splits=10,
                k = 25 
            )
print(f"underestimate_error using  {neighbours_size} strongest positively coorelated neighbours accounting for the personal  tendencies high/low  {underestimate_error} ")
underestimate_error_list.append((f"underestimate_error using 25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low",underestimate_error))



    


MAE using 25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low 0.5400134448817265
RMSE using 25 strongest positively coorelated neighbours  accounting for the personal  tendencies high/low0.6820267320787797
Ma at p=10 using 25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low 0.47314467607295896
overestimate_error using  25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low  0.5110370673816587 
underestimate_error using  50 strongest positively coorelated neighbours accounting for the personal  tendencies high/low  0.5572389916468498 


#### conclusions 
And it turned to be a very good idea, the mae decreased visibly and the RMse decreased  even more significantly from the previous best of around 0.79 to 0.68 now. The overestimate_error and underestimate error are very similar and significantly lower than they used to be what is also a very good outcome. The error at 10 best ratings also decreased, but only a tinge in comparison to its previous best value. Generally this idea seem to perform the best so far, as it minimizes the error in all the metrics, yielding it not only good but also versatile approach

### Item analysis
Lastly , why dont we analyze the items as well to improve our algorithm, the idea is that we do virtually the same as in the previous algorithm , however at the end of calculating this weighted deviation for our (userId,movie) tupple, we check if the score was equivocal, i.e if the score was close to the  line of change , to clarify lets think about an example: lets say average weighted score says 3.2  then it would be rounded to 3 because this is the closest viable rating to the result, but on the other hand is also not that far from 3.5  .In this case we check for the mean value for this item(movie) if it is above 3.5 we set our average to 3.5. Analogously we can round down with this approach. It seems  a great idea because it adds the item analysis aspect to the algorithm in the places where the user behaviour analysis is  torn between two ratings.

In [275]:
mae = evaluate_recommendation_quality_general(
            transformed_df=transformed_df,
            num_splits=10,
            predict_function=predict_scores_with_item_correction,  # Function to predict scores
            userId=610,
            k = 25
             # Pass the threshold as a keyword argument
        )
rmse = evaluate_recommendation_quality_rmse_general(
                transformed_df=transformed_df,
                predict_function=predict_scores_with_item_correction,  # Function to predict scores
                userId=610,
                num_splits=10,
                k = 25
            )
print(f"MAE using 25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low and items specificity {mae}")
mae_list.append((f"MAE using 25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low and items specificity",mae))
        
print(f"RMSE using 25 strongest positively coorelated neighbours  accounting for the personal  tendencies high/low{rmse}")
rmse_list.append((f"Rmse using 25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low and items specificity",rmse))

p_error = evaluate_recommendation_quality_at_p(
                p=10,
                transformed_df=transformed_df,
                num_splits=10,
                predict_function=predict_scores_with_item_correction,  # Function to predict scores
                userId=610,
                k = 25
            )
print(f"Ma at p=10 using 25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low and items specificity {p_error}")
perror_list.append((f"p_error using 25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low and items specificity",p_error))

overestimate_error = evaluate_recommendation_quality_general_weighted(
                transformed_df=transformed_df,
                weight_under=0,
                weight_over=1,
                userId=610,
                num_splits=10,
                predict_function=predict_scores_with_item_correction,  # Function to predict scores
                k = 25
            )
print(f"overestimate_error using  25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low and items specificity  {overestimate_error} ")
overestimate_error_list.append((f"overestimate_error using 25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low and items specificity",overestimate_error))
underestimate_error = evaluate_recommendation_quality_general_weighted(
                transformed_df=transformed_df,
                weight_under=1,
                weight_over=0,
                predict_function=predict_scores_with_item_correction,  # Function to predict scores
                userId=610,
                num_splits=10,
                k = 25 
            )
print(f"underestimate_error using  25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low and items specificity {underestimate_error} ")
underestimate_error_list.append((f"underestimate_error using 25strongest positively coorelated neighbours accounting for the personal  tendencies high/low and items specificity",underestimate_error))





MAE using 25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low and items specificity 0.5357688482689216
RMSE using 25 strongest positively coorelated neighbours  accounting for the personal  tendencies high/low0.7071960837101245
Ma at p=10 using 25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low and items specificity 0.48492063492063486
overestimate_error using  25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low and items specificity  0.7553065406303409 
underestimate_error using  25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low and items specificity 0.4248038154063024 


#### conclusions
The difference in comparison to the previous method is not very striking. The mae is slightly better here, however on the other hand the RMse is slightly  worse , also the mae at 10 best ratings is slightly worse. The underestimate_error is low as never before, meaning this system tends to  overestimate, and probably would be the best choices for user not being very picky but wanting to have a broad set of nice rated films to watch. In general the idea seem quite interesting but probably it requires more testing in order for it to make a good difference 

# Results comparison and conclusion

In [276]:
mae_list.sort(key = lambda x: x[-1])
rmse_list.sort(key = lambda x: x[-1])
underestimate_error_list.sort(key = lambda x: x[-1])
overestimate_error_list.sort(key = lambda x: x[-1])
perror_list.sort(key = lambda x: x[-1])

print ("MAE  scores:")
print(mae_list)
print('\n')
print ("rmse  scores:")
print(rmse_list)
print('\n')
print ("p_error at p =10   scores:")
print(perror_list)
print('\n')
print ("underestimate errror  scores:")
print(underestimate_error_list)
print('\n')
print ("overestimate error scores:")
print(overestimate_error_list)


MAE  scores:
[('MAE using 25 strongest positively coorelated neighbours accounting for the personal  tendencies high/low and items specificity', 0.5357688482689216), ('MAE using 25strongest positively coorelated neighbours accounting for the personal  tendencies high/low', 0.5400134448817265), ('MAE using 25 strongest positively coorelated neighbours', 0.5981621909345929), ('MAE using 50 strongest positively coorelated neighbours', 0.6003440458652951), ('MAE using 0% positive coorelation threshold', 0.6021952781071522), ('MAE using 15 strongest positively coorelated neighbours', 0.6031413821118339), ('MAE using 0% positive and  0.6%  negative  coorelation threshold', 0.604099744608393), ('MAE using 0% positive and  0.6%  negative  coorelation threshold', 0.6060534511840754), ('MAE using 0.2% positive coorelation threshold', 0.606241746389952), ('MAE using 0% positive and  0.6%  negative  coorelation threshold', 0.6113749808614168), ('MAE using 5 strongest positively coorelated neighbou

# Conclusions

- We believe that we tested pretty throughly several recommender system approaches, often building up one on the base of another . 
The main conclusion is that the system basing on the weighted average of deviation from means for 25 most correlated users performs the best .It is definitely the most versatile, performs good in all metrics, and also is simply the best for some metrics, for example MRSE - which is an important measure for this problem. 
- Additionaly, The item analysis is a very promising idea , because already here a simple approach with conflict resolutions basing on the mean for the given movie  used in case the weighted average of users gave equivocal score , proved promising . So probably after more work and testing some better results could be achieved using item-based recommender system combined with user-based. Regarding this matter, we have also tried to combine the user-based recommender based on correlation with the item-based recommender based on cosine-similarity between movies. However it turned out to be even more computationaly expensive than the user based recommender systems therefore we decided to resign of that.
- The baseline approach , very simple and cheap system, seems not to perform that bad in terms of mae. But for other, maybe more informative measures , it is clearly worse. Still it is not a bad idea to use it for comparison or as a baseline for predictions as it is not worse  by 'order of magnitude' than the other more sophisticated approaches
- Our approach to make use of the negatively coorelated users did not succed. Maybe the idea behind the algorithm, despite looking resonable,  was in fact invalid . Introducing this idea in our systems did only worsen performance in all of the metrics, However it may be only the issue of  invalid logic not issue of using the negative coorelation per se.